# Tutorial: Make an xarray Dataset CF-1.12 compliant

This notebook shows how to use `nc-check` to:
- inspect CF compliance with `ds.check.check_compliant()`
- apply safe fixes with `ds.check.make_cf_compliant()`

The object stays an `xarray.Dataset` throughout.


## 1. Setup


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import xarray as xr

import nc_check  # noqa: F401

## 2. Build a dataset with missing CF metadata


In [20]:
ds = xr.open_dataset(
    "/Users/luke/Downloads/GCB-2025_dataprod_UExP-FNN-U_1980-2024.nc", decode_cf=True
)[["fgco2", "sfco2", "area"]]

In [24]:
ds.check.make_compliant().check.compliance(engine="heuristic")

'<!DOCTYPE html>\n<html lang=\'en\'>\n<head>\n<meta charset=\'utf-8\'>\n<meta name=\'viewport\' content=\'width=device-width, initial-scale=1\'>\n<title>CF Compliance Report</title>\n<link href=\'https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css\' rel=\'stylesheet\'>\n<link rel=\'preconnect\' href=\'https://fonts.googleapis.com\'>\n<link rel=\'preconnect\' href=\'https://fonts.gstatic.com\' crossorigin>\n<link href=\'https://fonts.googleapis.com/css2?family=Manrope:wght@400;500;600;700&family=Fraunces:opsz,wght@9..144,600;9..144,700&display=swap\' rel=\'stylesheet\'>\n<style>\n:root{\n  --bg-top:#f4f8f6;\n  --bg-mid:#ecf5f4;\n  --bg-bottom:#f8fbfa;\n  --panel-bg:#ffffff;\n  --panel-border:#d1ddd8;\n  --panel-shadow:0 14px 35px rgba(15, 23, 42, .08);\n  --text-main:#122027;\n  --text-muted:#45606c;\n  --heading:#14252d;\n  --hero-accent-a:#0d9488;\n  --hero-accent-b:#f59e0b;\n  --header-bg:#eef4f2;\n  --header-bg-strong:#e2eeeb;\n  --radius-md:10px;\n  --radius-lg:

## 3. Check compliance before coercion


In [9]:
issues = ds.check.compliance(
    domain="ocean", report_format="html", report_html_file="output.html"
)
issues

'<!DOCTYPE html>\n<html lang=\'en\'>\n<head>\n<meta charset=\'utf-8\'>\n<meta name=\'viewport\' content=\'width=device-width, initial-scale=1\'>\n<title>CF Compliance Report</title>\n<link href=\'https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css\' rel=\'stylesheet\'>\n<link rel=\'preconnect\' href=\'https://fonts.googleapis.com\'>\n<link rel=\'preconnect\' href=\'https://fonts.gstatic.com\' crossorigin>\n<link href=\'https://fonts.googleapis.com/css2?family=Manrope:wght@400;500;600;700&family=Fraunces:opsz,wght@9..144,600;9..144,700&display=swap\' rel=\'stylesheet\'>\n<style>\n:root{\n  --bg-top:#f4f8f6;\n  --bg-mid:#ecf5f4;\n  --bg-bottom:#f8fbfa;\n  --panel-bg:#ffffff;\n  --panel-border:#d1ddd8;\n  --panel-shadow:0 14px 35px rgba(15, 23, 42, .08);\n  --text-main:#122027;\n  --text-muted:#45606c;\n  --heading:#14252d;\n  --hero-accent-a:#0d9488;\n  --hero-accent-b:#f59e0b;\n  --header-bg:#eef4f2;\n  --header-bg-strong:#e2eeeb;\n}\n\n*,\n*::before,\n*::after{\n  b

## 4. Apply compliance fixes


In [31]:
fixed = ds.check.make_cf_compliant()
fixed["temperature"].attrs["standard_name"] = "sea_surface_temperature"
fixed["temperature"].attrs["units"] = "degC"

print(type(fixed))
print("Conventions:", fixed.attrs.get("Conventions"))
print("lat attrs:", fixed["lat"].attrs)
print("lon attrs:", fixed["lon"].attrs)
print("time attrs:", fixed["time"].attrs)

fixed

<class 'xarray.core.dataset.Dataset'>
Conventions: CF-1.12
lat attrs: {'standard_name': 'latitude', 'long_name': 'latitude', 'units': 'degrees_north', 'axis': 'Y'}
lon attrs: {'standard_name': 'longitude', 'long_name': 'longitude', 'units': 'degrees_east', 'axis': 'X'}
time attrs: {'standard_name': 'time', 'axis': 'T'}


<xarray.Dataset> Size: 264B
Dimensions:      (time: 2, lat: 3, lon: 4)
Coordinates:
  * time         (time) int64 16B 0 1
  * lat          (lat) float64 24B 10.0 20.0 30.0
  * lon          (lon) float64 32B 100.0 110.0 120.0 130.0
Data variables:
    temperature  (time, lat, lon) float64 192B 0.8052 0.5414 ... 0.6556 0.5747
Attributes:
    Conventions:  CF-1.12

## 5. Re-check after coercion


In [32]:
issues_after = fixed.check.check_compliant(pretty_print=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ CF Compliance Report                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CF version     CF-1.12               
Engine         cfchecker             
Engine status  unavailable           
Counts         fatal=0 error=0 warn=0

          Coordinate Findings          
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Scope ┃ Severity ┃ Detail           ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ time  │ INFO     │ coord_attr:units │
└───────┴──────────┴──────────────────┘

                                         Suggested Improvements                                          
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Variable    ┃ Suggestion                                                                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ temperature │ {'units_check': {'status': 'mismatch', 'current_units': 'degC', 'expected_units': 'K'}} │
└─────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

                             Notes                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Note                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ cfchecker could not run; returned heuristic checks instead. │
└─────────────────────────────────────────────────────────────┘

╭───────────────────────────────────────────────── Checker Error ─────────────────────────────────────────────────╮
│ {'type': 'AttributeError', 'message': 'NetCDF: Attribute not found'}                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 6. Other dimension patterns


In [33]:
# time-only
only_time = xr.Dataset(
    data_vars={"signal": (("time",), [1.0, 2.0, 3.0])},
    coords={"time": [0, 1, 2]},
)

# lat-lon only
only_space = xr.Dataset(
    data_vars={"field": (("lat", "lon"), np.ones((2, 2)))},
    coords={"lat": [0.0, 1.0], "lon": [10.0, 11.0]},
)

print(
    "time-only conventions:",
    only_time.check.make_cf_compliant().attrs.get("Conventions"),
)
print(
    "space-only conventions:",
    only_space.check.make_cf_compliant().attrs.get("Conventions"),
)

time-only conventions: CF-1.12
space-only conventions: CF-1.12


## 7. Write to NetCDF (optional)

Once compliant, you can write normally:

```python
fixed.to_netcdf("example_cf.nc")
```
